## Calculate surface-forced watermass transformation
Presently, information about the products, e.g. names of variables is hard-coded. Could improve robustness by specifying generic names.

In [2]:
%load_ext autoreload
%autoreload 2
from dask.diagnostics import ProgressBar
import so_decadal_variability as so
import numpy as np
import xarray as xr
import os
import gsw
from xhistogram.xarray import histogram
import pandas as pd
import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from dask.distributed import Client
client = Client()
client

In [33]:
fluxname='merra2'
oceanname='en4'
onoceangrid=False
density = 'sigma0'
timeslice = slice('1980','2018')
dg = 0.02
density_edges = np.arange(1024,1028.5,dg)
save = True
savesequential = True
# Load data
ds = so.process.loaddata(fluxname,oceanname,timeslice,onoceangrid)
# Get grid
if onoceangrid:
    gridname = oceanname
else:
    gridname = fluxname
ds, xgrid = so.grid.get_xgcm(ds)
# Calculate a density variable
ds['sigma0'] = gsw.sigma0(ds['sa'].isel(depth=0).squeeze(),ds['sst'])
ds['b'] = xr.ones_like(ds['sigma0'])
ds['vol4d'] = ds['cell_area']*ds['dz']
ds = ds.isel(depth=0).squeeze().drop(['depth'])
ds = ds.chunk(chunks={'time':12})
# Calculate WMT
G = so.wmt.calc_watermasstransformation(ds,xgrid,density_edges,density=density,b_ones=False,penetrative_sw=False)

# Save
years = pd.to_datetime(G['time'].values).year.unique()
if save:
    if onoceangrid:
        localdir = '../../data/processed/'+fluxname+'_'+oceanname+'_onoceangrid'
    else:
        localdir = '../../data/processed/'+fluxname+'_'+oceanname
        
    if ~os.path.isdir(localdir):
        !mkdir $localdir
        
    if savesequential:
        for year in years:
            temp = G.sel(time=str(year))
            filename = 'G.'+density+'.'+str(dg)+'.'+str(year)
            print(filename)
            temp.to_netcdf(localdir+'/'+filename+'.nc')
    else:
        filename = 'G.'+density+'.'+str(dg)
        G.to_netcdf(localdir+'/'+filename+'.nc')

mkdir: cannot create directory ‘../../data/processed/merra2_en4’: File exists
G.sigma0.0.02.1980
G.sigma0.0.02.1981
G.sigma0.0.02.1982
G.sigma0.0.02.1983
G.sigma0.0.02.1984
G.sigma0.0.02.1985
G.sigma0.0.02.1986
G.sigma0.0.02.1987
G.sigma0.0.02.1988
G.sigma0.0.02.1989
G.sigma0.0.02.1990
G.sigma0.0.02.1991
G.sigma0.0.02.1992
G.sigma0.0.02.1993
G.sigma0.0.02.1994
G.sigma0.0.02.1995
G.sigma0.0.02.1996
G.sigma0.0.02.1997
G.sigma0.0.02.1998
G.sigma0.0.02.1999
G.sigma0.0.02.2000
G.sigma0.0.02.2001
G.sigma0.0.02.2002
G.sigma0.0.02.2003
G.sigma0.0.02.2004
G.sigma0.0.02.2005
G.sigma0.0.02.2006
G.sigma0.0.02.2007
G.sigma0.0.02.2008
G.sigma0.0.02.2009
G.sigma0.0.02.2010
G.sigma0.0.02.2011
G.sigma0.0.02.2012
G.sigma0.0.02.2013
G.sigma0.0.02.2014
G.sigma0.0.02.2015
G.sigma0.0.02.2016
G.sigma0.0.02.2017
G.sigma0.0.02.2018
